# WELCOME to the Tranformations and Deduplications of BENGAL-DATA.com

## Imports, Functions, Lists

In [13]:
import random
import time
import json
import datetime
import pandas as pd
import numpy as np

### Jaccard

In [2]:
# zakladny jaccard
def jaccard(str1, str2):
    set1 = set(str1)
    set2 = set(str2)
    inter = len(set1 & set2)
    union = len(set1 | set2)
    return inter / union
# obojsmerny jacacard
def bidirectional_jaccard(str1, str2):
    set1 = set(str1)
    set2 = set(str2)

    intersection1 = len(set1 & set2)
    intersection2 = len(set2 & set1)

    union = len(set1 | set2)

    return (intersection1 + intersection2) / (2 * union)
def jaccard_var(str1, str2):
    set1 = set(str1)
    set2 = set(str2)

    longest = max(str1, str2)
    first_intersections = set1 & set2

    return len(first_intersections) / (len(longest) + len(first_intersections))


### Jaro and Jaro-Winkler

In [3]:
# zakladny jaro
def jaro_distance(s1, s2):
    if s1 == s2:
        return 1.0
    len1 = len(s1)
    len2 = len(s2)
    if len1 == 0 or len2 == 0:
        return 0.0
    max_dist = (max(len(s1), len(s2)) // 2) - 1
    match = 0
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
    for i in range(len1):
        for j in range(max(0, i - max_dist), min(len2, i + max_dist + 1)):
            if s1[i] == s2[j] and hash_s2[j] == 0:
                hash_s1[i] = 1
                hash_s2[j] = 1
                match += 1
                break
    if match == 0:
        return 0.0
    t = 0
    point = 0
    for i in range(len1):
        if hash_s1[i]:
            while hash_s2[point] == 0:
                point += 1
            if s1[i] != s2[point]:
                point += 1
                t += 1
            else:
                point += 1
        t /= 2
    return (match / len1 + match / len2 + (match - t) / match) / 3.0


# zakladny jaro winkler
def jaro_winkler(str1, str2):
    jaro_dist = jaro_distance(str1, str2)
    if jaro_dist > 0.7:
        prefix = 0

        # Vypočet prefixovej zhodnosti

        for i in range(min(len(str1), len(str2))):
            if str1[i] == str2[i]:
                prefix += 1
            else:
                break
        prefix = min(4, prefix)

        jaro_dist += 0.1 * prefix * (1 - jaro_dist)
    return jaro_dist

### Find duplicates function

In [4]:
def similarity(data_par1, data_par2, approved_attributes, treshold):
    inc = 0
    if data_par1 is not None and data_par2 is not None:
        set_1 = approved_attributes
        set_2 = set(data_par1.keys()) & set(data_par2.keys())
        common_attributes = [x for x in set_1 if x in set_2]
        namesSimilarity = 0
        for attr in common_attributes:
            attr_value1 = data_par1[attr]
            attr_value2 = data_par2[attr]

            if isinstance(attr_value1, str) and isinstance(attr_value2, str):
                # Calculate similarity for string attributes
                if attr == "reg":
                    # Extract and compare only numeric parts of "reg" attribute
                    reg_num1 = "".join(re.findall(r"[0-9]", attr_value1))
                    reg_num2 = "".join(re.findall(r"[0-9]", attr_value2))
                    similarity_score = jaro_winkler(reg_num1, reg_num2)
                else:
                    # Calculate Jaro-Winkler similarity for other string attributes

                    #Jaccard + jaro, s predpokladom toho ze poddobne retazce by mali mat cca rovnaku sadu znakov
                    """similarity_score2 = jaccard(attr_value1, attr_value2)
                    similarity_score = jaro_winkler(attr_value1, attr_value2) + similarity_score2"""

                    similarity_score = jaro_winkler(attr_value1, attr_value2)

                #print("--------------------------------")
                #print(attr_value1)
                #print(attr_value2)
                if attr == "reg":
                    if similarity_score == 1:
                        #print("Rovnaké registračné číslo, ide o duplikát.")
                        inc = 1
                        break
                    else:
                        #print(f"Rozdielne registračné číslo.")
                        if(len(reg_num1) != 0 and len(reg_num2) != 0):
                            break
                        else:
                            #print("Nenajdené reg. číslo.")
                            continue
                if similarity_score < treshold:
                    #print(f"Tahle číča nie je rovnaká v atribúte '{attr}' s podobnosťou {similarity_score:.2f}")
                    o = 0
                else:
                    #print(f"Tahle číča je rovnaká v atribúte '{attr}' s podobnosťou {similarity_score:.2f}.")
                    if(attr == 'dam' or attr == 'sire' or attr == 'name'):
                        namesSimilarity += 1

            elif isinstance(attr_value1, list) and isinstance(attr_value2, list):
                # Calculate Jaccard similarity for list attributes
                similarity_score = jaccard(attr_value1, attr_value2)

                #print("*------------------------------*")
                #print(attr_value1)
                #print(attr_value2)
                if similarity_score < treshold:
                    #print(f"Tahle číča nie je rovnaká v atribúte '{attr}' s podobnosťou {similarity_score:.2f}")
                    o = 0
                else:
                    #print(f"Tahle číča je rovnaká v atribúte '{attr}' s podobnosťou {similarity_score:.2f}")
                    if(attr == "dame" or attr == "sire" or attr == "name"):
                        namesSimilarity += 1


            if(namesSimilarity >= 3):
                #print(f"Tento záznam číči je duplicát.")
                inc = 1
                break
    return inc

### Sub-functions

In [5]:
def filter_attributes(data_all, approved_attributes):
    filtered_data = []
    for cat_json in data_all:
        filtered_cat = {key: value for key, value in cat_json.items() if key in approved_attributes}
        filtered_data.append(filtered_cat)
    return filtered_data

# Sample dataset
dataset = ["Abeceda Dongy Black Credit Elytra", "Another Example String", "Sorting Words Alphabetically"]

In [42]:
# Function to sort words within each string
def sort_words_alphabetically(sentence):
    words = sentence.str.split()  # Split the string into a list of words
    sorted_words = sorted(words)  # Sort the words alphabetically
    separator = ' ' 
    sorted_sentence = ' '.join(' '.join(l) for l in sorted_words)  # Join the sorted words back into a string
    return sorted_sentence

In [33]:
def save_cat(cats_list):
    print("Total cats: " + str(len(cats_list)))

    currentDateTime = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    fileName = f"cats_{currentDateTime}.json"

    with open(fileName, 'w', encoding='utf-8') as outfile:
        json.dump(cats_list, outfile, indent=4, ensure_ascii=False)
    print(f"Saved to {fileName}")

#### Lists ####

In [6]:
approved = ["cat","reg nr", "alternate reg nr", "sverak reg",
            "color","eye color","coatcolor",
            "sex", "breed","date of birth",
            "sire", "sire reg",
            "dam", "dam reg"]

## 1. Start

In [7]:
#sorted_dataset = filter_attributes(allCats, approved)
allCats = [] 
with open("bengal-data.json", 'r', encoding='utf-8') as infile:
    allCats = json.load(infile)
df = pd.DataFrame(allCats)

## 2. Next there are transformations

### 2.1 Choose only aproved collums

In [8]:
df = df[approved]
#df.head()

### 2.1 Rename or change types of columns

In [17]:
df.rename(columns = {'cat':'name'}, inplace = True)
df = df.astype({'name':'str', 'sire':'str','dam':'str',
                'reg nr':'str','alternate reg nr':'str','sverak reg':'str',
})

### 2.3 second transformation, filling of nan reg nums

In [9]:
df["alternate reg nr"].fillna(df["sverak reg"], inplace=True)
df["reg nr"].fillna(df["alternate reg nr"], inplace=True)

tm = df[(df["alternate reg nr"] == df["sverak reg"])]
df.loc[(df["alternate reg nr"] == df["sverak reg"]), "sverak reg"] = np.nan

tp = df.loc[(df["reg nr"] == df["alternate reg nr"])]
df.loc[(df["reg nr"] == df["alternate reg nr"]), "alternate reg nr"] = np.nan

In [10]:
df.loc[tm.index].head()

,cat,reg nr,alternate reg nr,sverak reg,color,eye color,coatcolor,sex,breed,date of birth,sire,sire reg,dam,dam reg
753,CH MUMINDALENS MIDAS,SBT 111010 005,(SE)SVERAKRX273539,NaN,BROWN (BLACK) SPOTTED TABBY,NaN,NaN,Male,BENGAL,2010-11-10,HALLMARK SILVERSTREAM MOONSTRING,SBT 080405 011,MUMINDALENS BEN KISLA,SBT 060205 042
783,MAINSTREET MYSTIC WIND,SBT 022810 001,(SE)SVERAKRX271117,NaN,BROWN (BLACK) SPOTTED TABBY,NaN,NaN,Female,BENGAL,2010-02-28,MAINSTREET WITHOUT WARNING,SBT 081408 024,RW SGC MAINSTREET RESTLESS SPIRIT,SBT 011608 019
822,ABSOLUTELY ELIZAS DARLING,SBT 022110 007,(SE)SVERAKRX270417,NaN,BROWN (BLACK) SPOTTED TABBY,NaN,NaN,Female,BENGAL,2010-02-21,SGC ABSOLUTELY CATEGORY FIVE,SBT 071607 019,ABSOLUTELY BLACK MAGIC WOMAN,SBT 090908 023
1017,LYNXLAND BAGHEERA,SBT 092813 012,(SE)SVERAKRX310009,NaN,BROWN (BLACK) SPOTTED TABBY,NaN,NaN,Male,BENGAL,2013-09-28,WILDSTYLE LUCIFER OF GOLDLEOPARD,SBT 011012 013,LYNXLAND ZORANA,SBT 092012 058
1194,SILVERSPIRIT'S LUNAR ECLIPS,(SE)SVERAKRX300093,NaN,NaN,BROWN (BLACK) SPOTTED TABBY,NaN,NaN,Female,BENGAL,2013-07-16,QUEENANNE GREAT BALLSOFFIRE,SBT 041108 032,BENGA SILVERSPIRITS MOON,(DK)FDLO158988


### 2.3 Deduplication

#### 2.3.1 Sorted sub-set of names

In [48]:
#1. create subset with only names, for sorting
sort_names = df.loc[:, ("name", "sire", "dam")]
sort_names.loc["name"] = sort_words_alphabetically(sort_names["name"])
sort_names.loc["sire"] = sort_words_alphabetically(sort_names["sire"])
sort_names.loc["dam"] = sort_words_alphabetically(sort_names["dam"])
#sort_names.head()

,name,sire,dam
0,ALOTASPOTS KING LEO,MILLWOOD LOTTY OF JUNGLEBOOK,MILLWOOD KODACHROME OF JUNGLEBOOK
1,ALOTASPOTS KING RUMIEN,MILLWOOD LOTTY OF JUNGLEBOOK,MILLWOOD MOONLIGHT MAGIC
2,ALOTASPOTS MAGIC SPOTS,MILLWOOD LOTTY OF JUNGLEBOOK,MILLWOOD MOONLIGHT MAGIC
3,ALOTASPOTS MONA LISA,LIONSMOUNTAIN BENJAMIN,LIONSMOUNTAIN BONNIE
4,ALOTASPOTS TIGERR,MILLWOOD LOTTY OF JUNGLEBOOK,MILLWOOD MOONLIGHT MAGIC


#### 2.3.1 Accross detection of duplicates

In [ ]:
"""
    for i in range(len(sorted_data_all)):
        DUPLICATES = 0
        data_1 = sorted_data_all[i]
        data_1_ = dataAll[i]
        for ii in range(i+1, len(sorted_data_all)):  # for i in range(1000 - i):
            isDup = 0
            data_2 = sorted_data_all[ii]
            data_2_ = dataAll[ii]
            if data_1_["sex"] == data_2_["sex"] and data_1_["breed"] == data_2_["breed"]:
                isDup = similarity(data_1, data_2, approved, 0.9)
            if(isDup != 0):
                DUPLICATES += isDup
                print(F"Duplicates: {DUPLICATES}   indx1:{i} indx2:{ii}")
"""
# For pandas dataframe code -->
#conn_df = pd.concat([sort_names, df], axis=1)
#duplicates = df.loc[ similarity(conn_df, conn_df.shift(-1), approved, 0.9),:]

## 3. Save as json #

In [11]:
#df.to_json('file1.json',)
#df.to_json('file2.json', orient = 'split')
#df.to_json('file3.json', orient = 'split', compression = 'infer')
#df.to_json('file4.json', orient = 'split', compression = 'infer', index = 'true')

# Bin

In [12]:
#cats with all three colmns of reg full
#df[(df["reg nr"].notna() == True) & (df["alternate reg nr"].notna() == True) & (df["sverak reg"].notna() == True)]

#cats with all three colmns of reg null
#df[(df["reg nr"].notna() == False) & (df["alternate reg nr"].notna() == False) & (df["sverak reg"].notna() == False)]

#cats where reg nr is null
#df[(df["reg nr"].notna() == False)]